In [1]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
import asyncio

In [2]:
load_dotenv(override=True)

True

In [3]:
# basically system promts
instructions1 = "You are a sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."

In [4]:
sales_agent1 = Agent(
    name="Professional Sales Agent", 
    instructions=instructions1, 
    model="gpt-4o-mini")

sales_agent2 = Agent(
    name="Engaging Sales Agent", 
    instructions=instructions2, 
    model="gpt-4o-mini")

sales_agent3 = Agent(
    name="Busy Sales Agent", 
    instructions=instructions3, 
    model="gpt-4o-mini") 

In [7]:
result = Runner.run_streamed(sales_agent1, input="Write a cold sales email")
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

Subject: Simplify Your SOC 2 Compliance with AI-Driven Solutions

Hi [Recipient's Name],

I hope this message finds you well. I’m reaching out to introduce you to ComplAI, a cutting-edge SaaS solution designed to streamline SOC 2 compliance and simplify audit preparations.

Navigating the complexities of compliance can be daunting, and we understand that time and resources are often stretched thin. Our AI-powered platform is tailored to ensure that your organization not only meets the stringent requirements of SOC 2 but also does so with efficiency and ease.

Key benefits of using ComplAI include:

- **Automated Compliance Tracking**: Stay ahead of requirements with real-time monitoring and alerts.
- **Streamlined Documentation**: Effortlessly manage and organize your compliance documents in one secure location.
- **Audit Readiness**: Ensure you are always prepared for audits with comprehensive reporting capabilities.

We’ve already helped many organizations like yours reduce complianc

In [9]:
message = "Write a cold sales email"

with trace("Parallel cold emails"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message)
    )
outputs = [result.final_output for result in results]

for output in outputs:
    print(output + "\n\n")

Subject: Streamline Your SOC 2 Compliance Process with ComplAI

Hi [Recipient's Name],

I hope this message finds you well. My name is [Your Name], and I’m reaching out to introduce you to ComplAI, a cutting-edge SaaS solution designed to simplify SOC 2 compliance and audit preparations. 

In today’s fast-paced environment, maintaining compliance can be both time-consuming and complex. Our AI-powered platform automates key compliance processes, helping businesses like yours save time, reduce risk, and enhance overall efficiency. 

Some key benefits of using ComplAI include:
- **Automated Compliance Tracking:** Keep your compliance efforts organized and up-to-date effortlessly.
- **Audit Readiness:** Streamline your preparation to ensure you’re ready for audits when they occur.
- **Risk Management:** Identify and mitigate potential compliance risks proactively.

I would love to discuss how ComplAI can specifically benefit your organization and support your compliance objectives. Are you

In [ ]:
# now create a last agent that will pick the best cold sales email from the given options
# Orchestrator pattern:
# first 3 agent produce cold sales emails and then the 4th agent picks the best one
sales_picker = Agent(
    name="sales_picker",
    instructions="You pick the best cold sales email from the given options. \
Imagine you are a customer and pick the one you are most likely to respond to. \
Do not give an explanation; reply with the selected email only.",
    model="gpt-4o-mini"
)

message = "Write a cold sales email!"

with trace("Selection from sales people"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message)
    )

    outputs = [result.final_output for result in results]

    emails = "Cold sales emails:\n\n".join(outputs)
    best = await Runner.run(sales_picker, emails)

    print(f"best sales email:\n{best.final_output}")

best sales email:
Subject: Simplify Your SOC 2 Compliance with AI-Powered Solutions

Hi [Recipient's Name],

I hope this message finds you well. My name is [Your Name], and I'm with ComplAI, where we specialize in simplifying the SOC 2 compliance process through our innovative AI-driven SaaS platform.

Navigating the complexities of compliance can be a daunting task, especially when preparing for audits. Our solution is designed to streamline this process, reducing the time and effort required while ensuring that your organization meets the highest standards of data security and trust.

Benefits of our platform include:

- **Automated Documentation:** Generate and manage compliance documents effortlessly.
- **Real-Time Monitoring:** Stay updated with ongoing compliance status, mitigating risks proactively.
- **Audit Readiness:** Equip yourself with the tools necessary for a seamless audit experience.

Many of our clients have significantly reduced their compliance timelines and costs w

In [11]:
# add tools to the agent
@function_tool
def send_email(body: str):
    """ Send out an email with the given body to all sales prospects """
    sg = sendgrid.SendGridAPIClient(api_key=os.getenv("SENDGRID_API_KEY"))
    from_email = Email("gtimokhinas@proton.me")
    to_email = To("gtimokhina@gmail.com")
    subject = "Sales Email"
    content = Content("text/plain", body)
    mail = Mail(from_email, to_email, subject, content).get()
    response = sg.client.mail.send.post(request_body=mail)
    return {"status":"success"}

send_email


FunctionTool(name='send_email', description='Send out an email with the given body to all sales prospects', params_json_schema={'properties': {'body': {'title': 'Body', 'type': 'string'}}, 'required': ['body'], 'title': 'send_email_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x10b3ab9c0>, strict_json_schema=True, is_enabled=True)

Sales manager - the planning agent

In [13]:
# convert all agents into tools
description = "Write a cold sales email"

tool1 = sales_agent1.as_tool(tool_name = "sales_agent1", tool_description = description)
tool2 = sales_agent2.as_tool(tool_name = "sales_agent2", tool_description = description)
tool3 = sales_agent3.as_tool(tool_name = "sales_agent3", tool_description = description)

tools = [tool1, tool2, tool3, send_email]
tools

[FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x10b5ad440>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='sales_agent2', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x10b5ac040>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='sales_agent3', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required'

In [14]:
instructions ="You are a sales manager working for ComplAI. You use the tools given to you to generate cold sales emails. \
You never generate sales emails yourself; you always use the tools. \
You try all 3 sales_agent tools once before choosing the best one. \
You pick the single best email and use the send_email tool to send the best email (and only the best email) to the user."

sales_manager = Agent(name="Sales Manager", instructions=instructions, tools=tools, model="gpt-4o-mini")

message = "Send a cold sales email addressed to 'Dear CEO'"
with trace("Sales Manager"):
    result = await Runner.run(sales_manager, message)


Handoffs represent a way an agent can delegate to an agent, passing control to it

Handoffs and Agents-as-tools are similar:

In both cases, an Agent can collaborate with another Agent

With tools, control passes back

With handoffs, control passes across

In [15]:

subject_instructions = "You can write a subject for a cold sales email. \
You are given a message and you need to write a subject for an email that is likely to get a response."

html_instructions = "You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design."

subject_writer = Agent(name="Email subject writer", instructions=subject_instructions, model="gpt-4o-mini")
subject_tool = subject_writer.as_tool(tool_name="subject_writer", tool_description="Write a subject for a cold sales email")

html_converter = Agent(name="HTML email body converter", instructions=html_instructions, model="gpt-4o-mini")
html_tool = html_converter.as_tool(tool_name="html_converter",tool_description="Convert a text email body to an HTML email body")


In [16]:
@function_tool
def send_html_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body to all sales prospects """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("gtimokhinas@proton.me")  # Change to your verified sender
    to_email = To("gtimokhinas@proton.me")  # Change to your recipient
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

In [17]:
tools = [subject_tool, html_tool, send_html_email]
tools

[FunctionTool(name='subject_writer', description='Write a subject for a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'subject_writer_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x10b5d87c0>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='html_converter', description='Convert a text email body to an HTML email body', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'html_converter_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x10b5da7a0>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='send_html_email', description='Send out an email with the given subject and HTML body to all sales pros

In [18]:
instructions ="You are an email formatter and sender. You receive the body of an email to be sent. \
You first use the subject_writer tool to write a subject for the email, then use the html_converter tool to convert the body to HTML. \
Finally, you use the send_html_email tool to send the email with the subject and HTML body."


emailer_agent = Agent(
    name="Email Manager",
    instructions=instructions,
    tools=tools,
    model="gpt-4o-mini",
    handoff_description="Convert an email to HTML and send it")

In [22]:
tools = [tool1, tool2, tool3]
handoffs = [emailer_agent]
print(tools)
print(handoffs)

[FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x10b5ad440>, strict_json_schema=True, is_enabled=True), FunctionTool(name='sales_agent2', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x10b5ac040>, strict_json_schema=True, is_enabled=True), FunctionTool(name='sales_agent3', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': 

In [23]:
sales_manager_instructions = "You are a sales manager working for ComplAI. You use the tools given to you to generate cold sales emails. \
You never generate sales emails yourself; you always use the tools. \
You try all 3 sales agent tools at least once before choosing the best one. \
You can use the tools multiple times if you're not satisfied with the results from the first try. \
You select the single best email using your own judgement of which email will be most effective. \
After picking the email, you handoff to the Email Manager agent to format and send the email."

sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=handoffs,
    model="gpt-4o-mini"    
)

message = "Send out a cold sales email addressed to Dear CEO from Alice"

with trace("Automated SDR"):
    result = await Runner.run(sales_manager, message)